In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
table_path = "../data/data_num_325_reduce.xlsx"
#table_path = "../data/data_num_325_fea_291.xlsx"
data = pd.read_excel(table_path,index_col = 0)

y = data["Product_RON_loss"]
X = data.drop("Product_RON_loss",1)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-1.281018,0.218553,-1.720514,0.867571,-0.153856,0.018643,-1.008571,-1.880630,1.365240,-0.951186,...,-1.093848,0.282418,-0.948161,0.473711,-0.077370,-0.559532,0.273110,0.443778,0.458671,-0.237886
1,-1.191680,0.083281,-1.833636,0.998477,-0.410934,0.434378,-1.584774,-1.372637,0.867174,-1.442064,...,-0.257120,0.232324,-0.143356,0.553723,-0.497214,0.421631,1.508771,0.174567,0.752121,-1.471532
2,-1.110971,0.414777,-1.753464,0.462669,-0.318112,-0.033584,-1.442646,-1.807762,0.268393,-0.529419,...,-0.698237,0.866621,0.628230,0.170068,0.429665,-0.340125,0.235624,-1.174455,-0.190068,-0.449239
3,-1.046164,0.455359,-1.727485,0.490777,-0.270328,-0.010238,-1.628983,-1.619778,0.353826,-0.419763,...,-0.801899,0.717826,0.747057,-0.212439,-0.568275,0.405401,0.801241,-0.209465,-0.558158,0.013984
4,-1.072234,0.388213,-1.680629,0.132179,0.261363,0.093363,-1.042338,-1.964336,0.755776,0.575388,...,-0.263311,-0.219098,0.102339,1.118633,0.558476,0.053035,0.267638,-1.000021,0.132342,0.364713


In [3]:
# Split data into separate training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
#Try common machine learning
# R2 Score
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
def lets_try(train,labels):
    results={}
    def test_model(clf):
        
        cv = KFold(n_splits=5,shuffle=True,random_state=45)
        r2 = make_scorer(r2_score)
        r2_val_score = cross_val_score(clf, train, labels, cv=cv,scoring=r2)
        scores=[r2_val_score.mean()]
        return scores

    clf = linear_model.LinearRegression()
    results["Linear"]=test_model(clf)
    
    clf = linear_model.Ridge()
    results["Ridge"]=test_model(clf)
    
    clf = linear_model.BayesianRidge()
    results["Bayesian Ridge"]=test_model(clf)
    
    clf = linear_model.HuberRegressor()
    results["Hubber"]=test_model(clf)
    
    clf = linear_model.Lasso(alpha=1e-4)
    results["Lasso"]=test_model(clf)
    
    clf = BaggingRegressor()
    results["Bagging"]=test_model(clf)
    
    clf = RandomForestRegressor()
    results["RandomForest"]=test_model(clf)
    
    clf = AdaBoostRegressor()
    results["AdaBoost"]=test_model(clf)
    
    clf = svm.SVR()
    results["SVM RBF"]=test_model(clf)
    
    clf = svm.SVR(kernel="linear")
    results["SVM Linear"]=test_model(clf)
    
    results = pd.DataFrame.from_dict(results,orient='index')
    results.columns=["R Square Score"] 
    results=results.sort_values(by=["R Square Score"],ascending=False)
    results.plot(kind="bar",title="Model Scores")
    axes = plt.gca()
    axes.set_ylim([0.5,1])
    return results

lets_try(X,y)

/home/yuxinc/.conda/envs/tf22_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/yuxinc/.conda/envs/tf22_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/yuxinc/.conda/envs/tf22_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/yuxinc/.conda/envs/tf22_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in so

In [ ]:
#Try auto-sklearn

import autosklearn.regression
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_regression_example_tmp',
    output_folder='/tmp/autosklearn_regression_example_out',
)
automl.fit(X_train, y_train, dataset_name='boston')
print(automl.show_models())
predictions = automl.predict(X_test)
print("R2 score:", r2_score(y_test, predictions))

In [5]:
#Try Simple Neural Network
import tflearn
import tensorflow as tf
# Shape the labels
labels_nl = labels
labels_nl = labels_nl.reshape(-1,1)

tf.reset_default_graph()
r2 = tflearn.R2()
net = tflearn.input_data(shape=[None, train.shape[1]])
net = tflearn.fully_connected(net, 30, activation='linear')
net = tflearn.fully_connected(net, 10, activation='linear')
net = tflearn.fully_connected(net, 1, activation='linear')
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.01, decay_step=100)
net = tflearn.regression(net, optimizer=sgd,loss='mean_square',metric=r2)
model = tflearn.DNN(net)

model.fit(train, labels_nl,show_metric=True,validation_set=0.2,shuffle=True,n_epoch=50)

ModuleNotFoundError: No module named 'tensorflow.contrib'